# Setting Up

In [ ]:
%matplotlib inline

In [ ]:
! pip install -q transformers[sentencepiece] fastbook fastai nbdev plum-dispatch evaluate seqeval onnxruntime onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.7 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/msi1427/blurr.git
%cd blurr

Cloning into 'blurr'...
remote: Enumerating objects: 5063, done.
remote: Counting objects: 100% (890/890), done.
remote: Compressing objects: 100% (310/310), done.
remote: Total 5063 (delta 701), reused 699 (delta 573), pack-reused 4173 (from 1)
Receiving objects: 100% (5063/5063), 27.00 MiB | 12.17 MiB/s, done.
Resolving deltas: 100% (3932/3932), done.
/content/blurr


**Importing Necessary Libraries**

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *
from tqdm.notebook import tqdm
import numpy as np
from google.colab import drive
from sklearn import metrics
import onnxruntime as rt
from transformers import AutoTokenizer
import torch
from onnxruntime.quantization import quantize_dynamic, QuantType

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/MultiLabel_DataScience_Tags_Classification

/content/drive/MyDrive/MultiLabel_DataScience_Tags_Classification


# Data

In [ ]:
df = pd.read_csv("data/scraped/ques_details.csv")
df.head()

,title,description,tags,url
0,How to find significance for Gini coefficient changes?,"I'm using the Gini coefficient to evaluate the performance of a model. Making some changes (feature selection, hyperparameter tuning, etc.) I created variant models with different Gini coefficients.\nHow can I prove that the improvement in the Gini coefficient is indeed statistically significant?","['statistics', 'model-selection', 'gini-index']",https://datascience.stackexchange.com/questions/114955/how-to-find-significance-for-gini-coefficient-changes
1,How does batch normalization make a model less sensitive to hyperparameter tuning?,"Question 22 of 100+ Data Science Interview Questions and Answers for 2022 asks What is the benefit of batch normalization?\nThe first bullet of the answers to this is The model is less sensitive to hyperparameter tuning.\nThe wikipedia page batch normalization similarly claims:\nFurthermore, batch normalization seems to have a regularizing effect such that the network improves its generalization properties, and it is thus unnecessary to use dropout to mitigate overfitting.\nIn both cases I suspect they are referring to improved test error, with the former involving improved test error even...","['hyperparameter-tuning', 'batch-normalization', 'generalization']",https://datascience.stackexchange.com/questions/114953/how-does-batch-normalization-make-a-model-less-sensitive-to-hyperparameter-tunin
2,Time series data for Prophet model,"I have a time series data that has some missing dates. For example, in the dataset below '2017-08-06' is missing.\nTo enter this data in the Prophet model, do I have to create the data frame with all the missing dates? I know the Prophet model does not require the missing data for any specific time to be filled as it is a curve-fitting-based model. But I am unsure if all dates should be included for daily time series prediction. Thanks so much for answering my question.",['time-series'],https://datascience.stackexchange.com/questions/114950/time-series-data-for-prophet-model
3,Algorithm suggestion for correlated models,"I'm looking for suggestions on how to proceed with predicting on separate but correlated models.\nThe example I will use is housing data. I have three inputs:\nLatitude\nLongitude\n1-Google Street View Image\nI want to create models that predict things like:\nSquare Footage (regression)\nHome Price (regression)\nNumber of Floors (categorical)\nFor Sale or Not (binary)\nThe dataset is supervised, so we are given 7 features. The model in production will only have 3 inputs (lat, long, image). Ideally, we could solve this dependence by using 6 input features for every model, but production wil...",['model-selection'],https://datascience.stackexchange.com/questions/114948/algorithm-suggestion-for-correlated-models
4,How many matrix combinations is this? [closed],"Closed. This question is off-topic. It is not currently accepting answers.\nThis question does not appear to be about data science, within the scope defined in the help center.\nClosed 2 years ago.\nImprove this question\nInput Matrix:\n A B C D E \nA 1 - - - -\nB - 1 - - -\nC - - 1 - -\nD - - - 1 -\nE - - - - 1\n""-"": arbritary values\n""1"": locked ""1"" value (diagonally locked, ref below)\nOutput Matrix (one possible example after some arbitrary optimization based on the arbitrary values in the matrix):\n C E A D B \nC 1 - - - -\nE - 1 - - -\nA - - 1 - -\nD - - - 1 -\nB - - - - 1\nSidenot...",['matrix'],https://datascience.stackexchange.com/questions/114947/how-many-matrix-combinations-is-this


In [ ]:
df = df.dropna().reset_index(drop=True)
df.shape

(24500, 4)

In [ ]:
tags_list = df.tags.to_list()
tag_count = {}
for tags in tags_list:
  tag_list = eval(tags)
  for tag in tag_list:
    if tag in tag_count.keys():
      tag_count[tag] += 1
    else:
      tag_count[tag] = 1
print(f"Number of Tags: {len(tag_count)}")
print(tag_count)

threshold = int(len(df) * 0.001)
rare_tags = [key for key, value in tag_count.items() if value < threshold]
len(rare_tags), rare_tags[:5]

Number of Tags: 692
{'statistics': 744, 'model-selection': 170, 'gini-index': 2, 'hyperparameter-tuning': 172, 'batch-normalization': 43, 'generalization': 28, 'time-series': 1273, 'matrix': 43, 'python': 4414, 'predictive-modeling': 779, 'feature-engineering': 420, 'classification': 2262, 'binary-classification': 103, 'shap': 53, 'neural-network': 2964, 'rnn': 490, 'pytorch': 461, 'preprocessing': 361, 'machine-learning': 7629, 'svm': 405, 'linear-algebra': 60, 'vector-space-models': 28, 'nlp': 1776, 'text-generation': 57, 'tokenization': 47, 'decision-trees': 491, 'vc-theory': 18, 'pac-learning': 14, 'xgboost': 456, 'ensemble-modeling': 136, 'boosting': 76, 'multiclass-classification': 362, 'distribution': 103, 'lightgbm': 68, 'scikit-learn': 1554, 'logistic-regression': 448, 'regression': 1060, 'accuracy': 273, 'kernel': 61, 'seaborn': 56, 'transformer': 315, 'machine-translation': 57, 'huggingface': 62, 'dataset': 1041, 'active-learning': 22, 'deep-learning': 3209, 'clustering': 93

(393,
 ['gini-index', 'vc-theory', 'pac-learning', 'active-learning', 'annotation'])

In [ ]:
tags_list = df.tags.to_list()
revised_tag_list = []
indices_to_drop = []

for idx, tags in enumerate(tags_list):
  tag_list = eval(tags)
  revised_tags = []

  for tag in tag_list:
    if tag not in rare_tags:
      revised_tags.append(tag)

  if len(revised_tags) == 0:
    indices_to_drop.append(idx)
  else:
    revised_tag_list.append(revised_tags)

df = df.drop(indices_to_drop).reset_index(drop=True)
df['revised_tags'] = revised_tag_list
df.shape

(24340, 5)

In [ ]:
revised_tags_list = df.revised_tags.to_list()
revised_tag_count = {}
for tags in revised_tags_list:
  tag_list = tags
  for tag in tag_list:
    if tag in revised_tag_count.keys():
      revised_tag_count[tag] += 1
    else:
      revised_tag_count[tag] = 1
print(f"Number of Tags: {len(revised_tag_count)}")
print(revised_tag_count)

Number of Tags: 299
{'statistics': 744, 'model-selection': 170, 'hyperparameter-tuning': 172, 'batch-normalization': 43, 'generalization': 28, 'time-series': 1273, 'matrix': 43, 'python': 4414, 'predictive-modeling': 779, 'feature-engineering': 420, 'classification': 2262, 'binary-classification': 103, 'shap': 53, 'neural-network': 2964, 'rnn': 490, 'pytorch': 461, 'preprocessing': 361, 'machine-learning': 7629, 'svm': 405, 'linear-algebra': 60, 'vector-space-models': 28, 'nlp': 1776, 'text-generation': 57, 'tokenization': 47, 'decision-trees': 491, 'xgboost': 456, 'ensemble-modeling': 136, 'boosting': 76, 'multiclass-classification': 362, 'distribution': 103, 'lightgbm': 68, 'scikit-learn': 1554, 'logistic-regression': 448, 'regression': 1060, 'accuracy': 273, 'kernel': 61, 'seaborn': 56, 'transformer': 315, 'machine-translation': 57, 'huggingface': 62, 'dataset': 1041, 'deep-learning': 3209, 'clustering': 934, 'recommender-system': 322, 'similarity': 197, 'distance': 104, 'one-hot-en

In [ ]:
encode_tag_types = { key: idx for idx, (key, value) in enumerate(revised_tag_count.items())}
with open("deployment/tag_types_encoded.json", "w") as fp:
  json.dump(encode_tag_types, fp)

# We need this because for multilabel classification all tags have possibility to be present in the predictions
categorical_tag_list = []
revised_tags_list = df.revised_tags.to_list()

for revised_tags in revised_tags_list:
  categorical_list = [0] * len(encode_tag_types)
  for tag in revised_tags:
    tag_type_index = encode_tag_types[tag]
    categorical_list[tag_type_index] = 1
  categorical_tag_list.append(categorical_list)

df['tag_cat_list'] = categorical_tag_list
df.head()

,title,description,tags,url,revised_tags,tag_cat_list
0,How to find significance for Gini coefficient changes?,"I'm using the Gini coefficient to evaluate the performance of a model. Making some changes (feature selection, hyperparameter tuning, etc.) I created variant models with different Gini coefficients.\nHow can I prove that the improvement in the Gini coefficient is indeed statistically significant?","['statistics', 'model-selection', 'gini-index']",https://datascience.stackexchange.com/questions/114955/how-to-find-significance-for-gini-coefficient-changes,"[statistics, model-selection]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
1,How does batch normalization make a model less sensitive to hyperparameter tuning?,"Question 22 of 100+ Data Science Interview Questions and Answers for 2022 asks What is the benefit of batch normalization?\nThe first bullet of the answers to this is The model is less sensitive to hyperparameter tuning.\nThe wikipedia page batch normalization similarly claims:\nFurthermore, batch normalization seems to have a regularizing effect such that the network improves its generalization properties, and it is thus unnecessary to use dropout to mitigate overfitting.\nIn both cases I suspect they are referring to improved test error, with the former involving improved test error even...","['hyperparameter-tuning', 'batch-normalization', 'generalization']",https://datascience.stackexchange.com/questions/114953/how-does-batch-normalization-make-a-model-less-sensitive-to-hyperparameter-tunin,"[hyperparameter-tuning, batch-normalization, generalization]","[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
2,Time series data for Prophet model,"I have a time series data that has some missing dates. For example, in the dataset below '2017-08-06' is missing.\nTo enter this data in the Prophet model, do I have to create the data frame with all the missing dates? I know the Prophet model does not require the missing data for any specific time to be filled as it is a curve-fitting-based model. But I am unsure if all dates should be included for daily time series prediction. Thanks so much for answering my question.",['time-series'],https://datascience.stackexchange.com/questions/114950/time-series-data-for-prophet-model,[time-series],"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
3,Algorithm suggestion for correlated models,"I'm looking for suggestions on how to proceed with predicting on separate but correlated models.\nThe example I will use is housing data. I have three inputs:\nLatitude\nLongitude\n1-Google Street View Image\nI want to create models that predict things like:\nSquare Footage (regression)\nHome Price (regression)\nNumber of Floors (categorical)\nFor Sale or Not (binary)\nThe dataset is supervised, so we are given 7 features. The model in production will only have 3 inputs (lat, long, image). Ideally, we could solve this dependence by using 6 input features for every model, but production wil...",['model-selection'],https://datascience.stackexchange.com/questions/114948/algorithm-suggestion-for-correlated-models,[model-selection],"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
labels = list(encode_tag_types.keys())
len(labels), labels[:5]

(299,
 ['statistics',
  'model-selection',
  'hyperparameter-tuning',
  'batch-normalization',
  'generalization'])

# Data Split

In [ ]:
splitter = RandomSplitter(valid_pct=0.1, seed=42)
train_ids, valid_ids = splitter(df)
len(train_ids), len(valid_ids)

(21906, 2434)

In [ ]:
valid_df = df.loc[valid_ids]
valid_df.head()

,title,description,tags,url,revised_tags,tag_cat_list
6942,WGAN-GP slow critic training time,"I am implementing WGAN-GP using Tensorflow 2.0, but each training iteration of the critic is very slow (about 4 secs on my CPU, and somehow 9 secs on Colab GPU).\nIs WGAN-GP usually this slow or there is a flaw in my code?\nHere's my code for training the critic:\ndef train_critic(self, X_real, batch_size, gp_loss_factor, optimizer):\n y_real = np.ones((batch_size, 1))\n\n # Get batch of generated images\n noise = np.random.normal(0, 1, (batch_size, self.z_dim))\n X_fake = self.gen.predict(noise)\n y_fake = -np.ones((batch_size, 1))\n\n X = np.vstack((X_real, X_fake))\n ...","['tensorflow', 'gan', 'generative-models']",https://datascience.stackexchange.com/questions/60302/wgan-gp-slow-critic-training-time,"[tensorflow, gan, generative-models]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
11889,Default parameters for decision trees give better results than parameters optimised using GridsearchCV,"I am using Gridsearch for a DecisionTreeClassifier predicting a binary outcome. When I run fit and predict with default parameters, I get the following results:\nAccuracy: 0.9602242115860793\nF1: 0.9581087077004674\nThen I try GridsearchCV:\nfrom sklearn.model_selection import GridSearchCV\n\nparam_grid = {""criterion"": [""gini"", ""entropy""],\n ""min_samples_split"": [2, 10],\n ""max_depth"": [2, 5, 10]\n }\n\ngrid = GridSearchCV(dtc, param_grid, cv=3, scoring='neg_mean_squared_error')\n\n\ngrid.fit(X_train, y_train.values.ravel())\n\ny_pred_class = grid.pre...","['decision-trees', 'grid-search', 'gridsearchcv']",https://datascience.stackexchange.com/questions/40599/default-parameters-for-decision-trees-give-better-results-than-parameters-optimi,"[decision-trees, grid-search, gridsearchcv]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
3266,Reinforcement Learning in a game against itself?,Let's we have a tictactoe design using RL against a random player. We can describe the system by enhancing and giving rewards to good actions. But what if the Rl model is played with itself? What changes in this case than playing with a random player?,"['machine-learning', 'reinforcement-learning', 'q-learning']",https://datascience.stackexchange.com/questions/112733/reinforcement-learning-in-a-game-against-itself,"[machine-learning, reinforcement-learning, q-learning]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
9053,Select estimators for Voting classifier,"I don't know the efficient method to select the best subset of the estimator that saves time and highest accuracy. How to choose the best estimators for the Voting classifier?\nVoting Classifier is an ensemble technique for combines the predictions of several underlying machine learning models (estimators). It makes a final prediction based on either majority voting (""hard"") or averaging probabilities (""soft""). Example: I have 20 estimators. I want to select the subset of these estimators that achieve the highest accuracy by saving time.\ndataset: https://drive.google.com/file/d/1lcfDLwnJ1...","['machine-learning', 'python', 'predictive-modeling', 'ensemble-modeling', 'ensemble']",https://datascience.stackexchange.com/questi

We will be using `valid_df` for all inference testing

# Fastai & Blurr Inference

**Model Name**

In [ ]:
model_name="bert-base-uncased"

In [ ]:
model_path = f"models/{model_name}/question-classifier-stage-1.pkl"
learner_inf = load_learner(model_path)

In [ ]:
learner_inf.blurr_predict("How can I predict the best treatment to give to new patient?")[0]['labels']

['predictive-modeling', 'machine-learning']

## Evaluation

In [ ]:
from sklearn.metrics import precision_score, recall_score
def metric_measures(test_df, preds):

  targets = [np.asarray(target) for target in test_df['tag_cat_list'].to_list()]
  outputs = [np.asarray(pred) for pred in preds]


  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  precision_micro = precision_score(targets, outputs, average='micro')
  precision_macro = metrics.precision_score(targets, outputs, average='macro')
  recall_micro = metrics.recall_score(targets, outputs, average='micro')
  recall_macro = metrics.recall_score(targets, outputs, average='macro')

  print(f"Accuracy = {accuracy}")
  print(f"Precision (Micro) = {precision_micro}")
  print(f"Recall (Micro) = {recall_micro}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"Precision (Macro) = {precision_macro}")
  print(f"Recall (Macro) = {recall_macro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

  return

In [ ]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row['description']
  labels = learner_inf.blurr_predict(desc)[0]['labels']
  pred_tags = [0] * len(encode_tag_types)
  for label in labels:
    pred_tags[encode_tag_types[label]] = 1
  preds.append(pred_tags)

preds[0][:20]

  0%|          | 0/2434 [00:00<?, ?it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
metric_measures(valid_df, preds)

Accuracy = 0.07518488085456039
Precision (Micro) = 0.6552147239263804
Recall (Micro) = 0.29983155530600786
F1 Score (Micro) = 0.41140215716486905
Precision (Macro) = 0.2830715810283197
Recall (Macro) = 0.1387212835109546
F1 Score (Macro) = 0.17013646093311086


# Convert to ONNX

### ONNX
ONNX (Open Neural Network Exchange) is a open standard for representing machine learning models.
- It allows developers to move models between different frameworks (such as PyTorch, TensorFlow, and Caffe2) without losing performance or accuracy.
ONNX makes it easier to build and run AI models on a variety of hardware, including GPUs, CPUs, and custom accelerators.
- This standard helps eliminate the need for multiple conversion tools and provides a unified representation of the model across different tools and frameworks.
- ONNX is being developed by a collaboration of companies including Microsoft, Facebook, Amazon, and IBM, among others, making it a well-supported and widely-adopted standard.
- Converting to ONNX runtime often makes model faster.

In [ ]:
model_path = f"models/{model_name}/question-classifier-stage-1.pkl"
learner_inf = load_learner(model_path)

In [ ]:
learner_inf.model.hf_model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
classifier = learner_inf.model.hf_model.eval()

torch.onnx.export(
    classifier,
    torch.LongTensor([[0] * 512]),
    f'models/{model_name}/question-classifier.onnx',
    input_names=['input_ids'],
    output_names=['output'],
    opset_version=14,
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},
        'output': {0: 'batch_size'}
    }
)

In [ ]:
onnx_model_path = f'models/{model_name}/question-classifier.onnx'
quantized_onnx_model_path = f'models/{model_name}/question-classifier-quantized.onnx'

quantize_dynamic(
    onnx_model_path,
    quantized_onnx_model_path,
    weight_type=QuantType.QUInt8,
)

# ONNX Inference

## Normal ONNX

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

class_labels = list(encode_tag_types.keys())

inf_session = rt.InferenceSession(f'models/{model_name}/question-classifier.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_tags = [0] * len(encode_tag_types)
  for label in labels:
    pred_tags[encode_tag_types[label]] = 1
  preds.append(pred_tags)

  0%|          | 0/2434 [00:00<?, ?it/s]

In [ ]:
metric_measures(valid_df, preds)

Accuracy = 0.07107641741988496
Precision (Micro) = 0.6611709286675639
Recall (Micro) = 0.27582818641212803
F1 Score (Micro) = 0.3892630744849445
Precision (Macro) = 0.2812832498328307
Recall (Macro) = 0.1307781995673581
F1 Score (Macro) = 0.16433956438030564


## Quantized ONNX

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

class_labels = list(encode_tag_types.keys())

inf_session = rt.InferenceSession(f'models/{model_name}/question-classifier-quantized.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [ ]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_tags = [0] * len(encode_tag_types)
  for label in labels:
    pred_tags[encode_tag_types[label]] = 1
  preds.append(pred_tags)

  0%|          | 0/2434 [00:00<?, ?it/s]

In [ ]:
metric_measures(valid_df, preds) #

Accuracy = 0.05258833196384552
Precision (Micro) = 0.6975247524752475
Recall (Micro) = 0.19778214486243684
F1 Score (Micro) = 0.30818022747156604
Precision (Macro) = 0.22261532734276243
Recall (Macro) = 0.07471912149183565
F1 Score (Macro) = 0.1027218605534621
